# Next Pitch Prediction using GRUs (Tests)

After fine tuning the hyperparameters of the GRU model, we will now train models for each pitcher and evaluate the performance of the model using the test set.

In [1]:
import os
import tensorflow as tf

os.chdir('../..')
tf.keras.utils.set_random_seed(42)

In [2]:
def build_network(input_layer, num_targets, name='', num_hidden_units=128, num_hidden_layers=1, activation='tanh', recurrent_activation='sigmoid', reg=None, dropout=None):
    
    # Hidden layers
    x = input_layer
    for i in range(num_hidden_layers - 1):
        layer_name = f"{name}_hidden_{i+1}"
        if reg:
            x = tf.keras.layers.GRU(units=num_hidden_units, activation=activation, recurrent_activation=recurrent_activation, return_sequences=True, kernel_regularizer=reg, name=layer_name)(x)
        else:
            x = tf.keras.layers.GRU(units=num_hidden_units, activation=activation, recurrent_activation=recurrent_activation, return_sequences=True, name=layer_name)(x)
        if dropout:
            x = tf.keras.layers.Dropout(dropout, name=f"{layer_name}_dropout")(x)
    
    # Last hidden layer without return_sequences
    layer_name = f"{name}_hidden_{num_hidden_layers}"
    if reg:
        x = tf.keras.layers.GRU(units=num_hidden_units, activation=activation, recurrent_activation=recurrent_activation, return_sequences=False, kernel_regularizer=reg, name=layer_name)(x)
    else:
        x = tf.keras.layers.GRU(units=num_hidden_units, activation=activation, recurrent_activation=recurrent_activation, return_sequences=False, name=layer_name)(x)
    if dropout:
        x = tf.keras.layers.Dropout(dropout, name=f"{layer_name}_dropout")(x)

    # Output layer
    outputs = tf.keras.layers.Dense(units=num_targets, activation='softmax', name=f'{name}_output')(x)

    return outputs

In [3]:
def build_model(input_shape, num_pitches, num_vertical_locs, num_horizontal_locs):
    # Define the input layer
    input_layer = tf.keras.Input(shape=input_shape)

    pitch_output = build_network(input_layer, num_targets=num_pitches, name='pitch',
                         num_hidden_layers=3,
                         num_hidden_units=128,
                         activation='relu',
                         recurrent_activation='tanh')
    vertical_output = build_network(input_layer, num_targets=num_vertical_locs, name='vertical',
                         num_hidden_layers=1,
                         num_hidden_units=128,
                         activation='relu',
                         recurrent_activation='sigmoid')
    horizontal_output = build_network(input_layer, num_targets=num_horizontal_locs, name='horizontal',
                         num_hidden_layers=1,
                         num_hidden_units=128,
                         activation='relu',
                         recurrent_activation='sigmoid')
    # Combine the models
    ensemble_model = tf.keras.models.Model(inputs=input_layer, outputs=[pitch_output, vertical_output, horizontal_output])
    # Compile the model
    ensemble_model.compile(optimizer='adam',
                           loss={'pitch_output': 'categorical_crossentropy',
                                 'vertical_output': 'categorical_crossentropy',
                                 'horizontal_output': 'categorical_crossentropy'},
                           metrics=['accuracy', 'accuracy', 'accuracy'])
    return ensemble_model

In [4]:
from utils.callbacks import FreezeOutputCallback

freeze_output_callback = FreezeOutputCallback(patience=5)

In [5]:
from utils import preprocessing
from sklearn.model_selection import train_test_split


def train_test_model(pitcher):
    print(f'Training model for {pitcher}...')
    # Load the data
    X, y_pitch, y_vertical, y_horizontal = preprocessing.get_sequences(os.path.join('data', 'raw', f'{pitcher}.csv'))

    # Split the data into training, validation, and testing sets
    X_train, X_temp, y_pitch_train, y_pitch_temp, y_vertical_train, y_vertical_temp, y_horizontal_train, y_horizontal_temp = train_test_split(
        X, y_pitch, y_vertical, y_horizontal, test_size=0.4, random_state=54)

    X_val, X_test, y_pitch_val, y_pitch_test, y_vertical_val, y_vertical_test, y_horizontal_val, y_horizontal_test = train_test_split(
        X_temp, y_pitch_temp, y_vertical_temp, y_horizontal_temp, test_size=0.5, random_state=42)

    num_pitches = y_pitch.shape[1]
    num_vertical_locs = y_vertical.shape[1]
    num_horizontal_locs = y_horizontal.shape[1]

    # Build the model
    model = build_model(input_shape=(X_train.shape[1], X_train.shape[2]), num_pitches=num_pitches, num_vertical_locs=num_vertical_locs, num_horizontal_locs=num_horizontal_locs)

    # Train the model
    history = model.fit(X_train,
                        {'pitch_output': y_pitch_train,
                         'vertical_output': y_vertical_train,
                         'horizontal_output': y_horizontal_train},
                        epochs=100, batch_size=64,
                        validation_data=(X_val,
                                         {'pitch_output': y_pitch_val,
                                          'vertical_output': y_vertical_val,
                                          'horizontal_output': y_horizontal_val}),
                        callbacks=[freeze_output_callback],
                        verbose=0
                        )

    max_pitch_val_accuracy = max(history.history['val_pitch_output_accuracy'])
    max_vertical_val_accuracy = max(history.history['val_vertical_output_accuracy'])
    max_horizontal_val_accuracy = max(history.history['val_horizontal_output_accuracy'])

    test_loss, pitch_test_accuracy, vertical_test_accuracy, horizontal_test_accuracy = model.evaluate(X_test,
                                                                                                      {'pitch_output': y_pitch_test,
                                                                                                       'vertical_output': y_vertical_test,
                                                                                                       'horizontal_output': y_horizontal_test})
    
    return {'Pitcher': pitcher,
            'Pitch_Val_Acc': max_pitch_val_accuracy,
            'Vertical_Val_Acc': max_vertical_val_accuracy,
            'Horizontal_Val_Acc': max_horizontal_val_accuracy,
            'Pitch_Test_Acc': pitch_test_accuracy,
            'Vertical_Test_Acc': vertical_test_accuracy,
            'Horizontal_Test_Acc': horizontal_test_accuracy}
    

In [6]:
import pandas as pd

# List of pitchers
pitchers = ['blake_snell', 'corbin_burnes', 'dylan_cease', 'gerrit_cole']

# Initialize a dataframe to store the results
results_df = pd.DataFrame(columns=['Pitcher', 'Pitch_Val_Acc', 'Vertical_Val_Acc', 'Horizontal_Val_Acc', 'Pitch_Test_Acc', 'Vertical_Test_Acc', 'Horizontal_Test_Acc'])

results = []
for pitcher in pitchers:
    # Store the results in the dataframe
    results.append(train_test_model(pitcher))
    
results_df = pd.DataFrame(results)

Training model for blake_snell...

Freezing output pitch at 28 epochs.

Freezing output vertical at 35 epochs.

Freezing output horizontal at 51 epochs.

All outputs frozen. Stopping training at epoch 51.
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - horizontal_output_accuracy: 0.6480 - loss: 5.4767 - pitch_output_accuracy: 0.6129 - vertical_output_accuracy: 0.6595
Training model for corbin_burnes...

Freezing output vertical at 27 epochs.

Freezing output horizontal at 27 epochs.

Freezing output pitch at 29 epochs.

All outputs frozen. Stopping training at epoch 29.
217/217 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - horizontal_output_accuracy: 0.6084 - loss: 4.2230 - pitch_output_accuracy: 0.5899 - vertical_output_accuracy: 0.6989
Training model for dylan_cease...

Freezing output vertical at 27 epochs.

Freezing output pitch at 41 epochs.

Freezing output horizontal at 42 epochs.

All outputs frozen. Stopping training at epoch 42.
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - horizontal_output_ac

In [7]:
results_df.to_csv(os.path.join('lstm_test_results.csv'), index=False)
results_df

,Pitcher,Pitch_Val_Acc,Vertical_Val_Acc,Horizontal_Val_Acc,Pitch_Test_Acc,Vertical_Test_Acc,Horizontal_Test_Acc
0,blake_snell,0.624416,0.679526,0.648965,0.651136,0.626086,0.664663
1,corbin_burnes,0.590922,0.701009,0.599856,0.607637,0.598559,0.692363
2,dylan_cease,0.634842,0.619278,0.653414,0.660390,0.627174,0.616843
3,gerrit_cole,0.620874,0.610169,0.640351,0.631282,0.618495,0.621618
